# Import Libaries

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time

In [2]:
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [3]:
df

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0          1       0       0       0       0       0       0       0       0   
1          0       0       0       0       0       0       0       0       0   
2          1       0       0       0       0       0       0       0       0   
3          4       0       0       0       0       0       0       0       0   
4          0       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
41995      0       0       0       0       0       0       0       0       0   
41996      1       0       0       0       0       0       0       0       0   
41997      7       0       0       0       0       0       0       0       0   
41998      6       0       0       0       0       0       0       0       0   
41999      9       0       0       0       0       0       0       0       0   

       pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  \
0           0  ...         0         0         0         0         0   
1           0  ...         0         0         0         0         0   
2           0  ...         0         0         0         0         0   
3           0  ...         0         0         0         0         0   
4           0  ...         0         0         0         0         0   
...       ...  ...       ...       ...       ...       ...       ...   
41995       0  ...         0         0         0         0         0   
41996       0  ...         0         0         0         0         0   
41997       0  ...         0         0         0         0         0   
41998       0  ...         0         0         0         0         0   
41999       0  ...         0         0         0         0         0   

       pixel779  pixel780  pixel781  pixel782  pixel783  
0             0         0         0         0         0  
1             0         0         0         0         0  
2             0         0         0         0         0  
3             0         0         0         0         0  
4             0         0         0         0         0  
...         ...       ...       ...       ...       ...  
41995         0         0         0         0         0  
41996         0         0         0         0         0  
41997         0         0         0         0         0  
41998         0         0         0         0         0  
41999         0         0         0         0         0  

[42000 rows x 785 columns]

In [4]:
[col for col in df.columns if df[col].isnull().sum()>0]

[]

# Initial Evaluation

In [5]:
y = df["label"]
X = df.iloc[:, 1:]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=42,stratify=y)

In [7]:
clf = SVC()
start = time.time()
clf.fit(X_train, y_train)
end = time.time()
print(end - start)
y_pred = clf.predict(X_test)

130.06951570510864


In [8]:
accuracy_score(y_test, y_pred)

0.9718253968253968

#### 97% Accuracy. This model is too good to be true :D

In [9]:
preds = clf.predict(test)
submission.iloc[:, 1] = preds
submission.to_csv("submission.csv", index=False)

### As expected, This model does not perform well on unseen data :/

## Data Preprocessing

In [10]:
###Check whether the training data is biased or not
df["label"].value_counts()

1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: label, dtype: int64

### Data for label 1 is 4684(which is max) and for label 5 is 3795(which is min) meaning data is biased towards 1

In [13]:
#As the data is pixel data so its values varies from 0-255 we to standardize all the features
scalar = StandardScaler()
scalar.fit(X_train)
X_train = scalar.transform(X_train)
X_test =  scalar.transform(X_test)
test = scalar.transform(test)

In [14]:
clf = SVC(kernel='poly') ## I am using poly kernel since it is popular is image
start = time.time()
clf.fit(X_train, y_train)
end = time.time()
print(end - start)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

416.77183389663696


In [15]:
preds = clf.predict(test)
submission.iloc[:, 1] = preds
submission.to_csv("submission_01.csv", index=False)

### Phew using kernel = 'Poly' did the trick. In one shot we reached from 0 to 0.83771 accuracy

## Hyperparameter Tunning

In [16]:
clf = SVC(decision_function_shape='ovo', kernel='poly')
start = time.time()
clf.fit(X_train, y_train)
end = time.time()
print(end - start)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

422.36673855781555
[3 0 8 ... 8 0 3]


0.8326190476190476

In [18]:
preds = clf.predict(test)
submission.iloc[:, 1] = preds
submission.to_csv("submission_02.csv", index=False)

#### using OVO as decision_function_shape decreased accuracy of the model, i will not be using this one